## Penerapan Algortima A* Search Dalam Kasus Pencarian Rute Tercepat dan Biaya Termurah dari Kota Bandar Lampung Menuju Medan 

- **Nama:** Firman Farel Richardo
- **NPM:** 2315061099
- **Kelas:** PSTI-C
- **Email:** farelpasaribu04@gmail.com

    **Universitas Lampung** 

### Memuat dan Mempersiapkan Data

In [16]:
import pandas as pd
import heapq

In [17]:
try:
    df = pd.read_csv('lampungmedan.csv')

except FileNotFoundError:
    print("Pastikan file dataset berada di folder yang sama dengan script ini.")
    exit()
    
print(df)

                Asal            Tujuan         Jalur  Jarak (km)  \
0            Lampung         Palembang         Timur         360   
1            Lampung          Baturaja        Tengah         230   
2            Lampung              Krui         Barat         245   
3          Palembang             Jambi         Timur         280   
4          Palembang     Lubuk Linggau        Tengah         320   
5           Baturaja     Lubuk Linggau        Tengah         215   
6               Krui          Bengkulu         Barat         260   
7              Jambi         Pekanbaru         Timur         395   
8              Jambi       Bukittinggi        Tengah         450   
9      Lubuk Linggau       Bukittinggi        Tengah         430   
10          Bengkulu            Padang         Barat         460   
11         Pekanbaru     Rantau Prapat         Timur         380   
12       Bukittinggi  Padang Sidempuan        Tengah         230   
13            Padang  Padang Sidempuan         B

### Membuat struktur graf dari DataFrame pandas

In [18]:
def create_graph_from_csv(file_path):
    """
    Fungsi untuk membaca file CSV dan mengubahnya menjadi struktur data Graph.
    Graph direpresentasikan sebagai dictionary.
    """
    graph = {}
    try:
        # Baca file CSV menggunakan pandas
        df = pd.read_csv(file_path)

        # Iterasi setiap baris dalam dataframe (file csv)
        for _, row in df.iterrows():
            asal = row['Asal']
            tujuan = row['Tujuan']
            
            # Buat dictionary untuk menyimpan detail perjalanan (edge)
            # Pastikan tipe data dikonversi ke numerik (float/int)
            data = {
                'distance': float(row['Jarak (km)']),
                'time': float(row['Estimasi Waktu (jam)']),
                'cost': float(row['Estimasi Biaya Tol (Rp)'])
            }

            # Jika kota 'Asal' belum ada di graph, inisialisasi dengan list kosong
            if asal not in graph:
                graph[asal] = []
            
            # Tambahkan rute (tujuan dan datanya) ke kota 'Asal'
            graph[asal].append((tujuan, data))

        # Pastikan semua kota (termasuk yang hanya menjadi tujuan) ada di dalam keys graph
        # Ini penting agar algoritma A* dapat berjalan dengan benar
        all_cities = set(df['Asal']).union(set(df['Tujuan']))
        for city in all_cities:
            if city not in graph:
                graph[city] = []
        
        print("✅ Berhasil memuat data dari CSV dan membuat graph.")
        return graph

    except FileNotFoundError:
        print(f"❌ ERROR: File '{file_path}' tidak ditemukan. Pastikan file berada di direktori yang sama.")
        return None
    except KeyError as e:
        print(f"❌ ERROR: Kolom {e} tidak ditemukan di file CSV. Pastikan nama kolom sesuai (Asal, Tujuan, dll.).")
        return None

### Heuristik (Estimasi waktu ke tujuan dari setiap kota)

In [19]:
heuristics = {
    'Lampung': 15.0,
    'Palembang': 11.0,
    'Baturaja': 13.0,
    'Krui': 15.0,
    'Jambi': 8.0,
    'Lubuk Linggau': 10.0,
    'Bengkulu': 12.0,
    'Pekanbaru': 5.0,
    'Bukittinggi': 7.0,
    'Padang': 9.0,
    'Rantau Prapat': 3.5,
    'Padang Sidempuan': 6.0,
    'Medan': 0
}

### Kode Program A* Search

In [20]:
def a_star_search(graph, start, goal, cost_type='time'):
    """
    Fungsi A* Search untuk mencari rute optimal.
    Logika fungsi ini SAMA SEPERTI SEBELUMNYA, tidak ada perubahan.
    """
    open_set = [(0, start, [])]
    g_score = {city: float('inf') for city in graph}
    g_score[start] = 0
    
    while open_set:
        _, current, path = heapq.heappop(open_set)
        current_path = path + [current]

        if current == goal:
            total_cost = g_score[goal]
            return current_path, total_cost

        for neighbor, data in graph.get(current, []):
            tentative_g_score = g_score[current] + data[cost_type]

            if tentative_g_score < g_score[neighbor]:
                g_score[neighbor] = tentative_g_score
                h_score = heuristics.get(neighbor, 0) if cost_type == 'time' else 0
                f_score = tentative_g_score + h_score
                heapq.heappush(open_set, (f_score, neighbor, current_path))
                
    return "Rute tidak ditemukan", float('inf')

### Eksekusi dan Hasil Algoritma A* Search

In [21]:
# =====================================================================
# BAGIAN 1: FUNGSI BANTUAN (Letakkan ini setelah fungsi a_star_search)
# =====================================================================

def get_path_details(path, graph):
    """
    Fungsi untuk menghitung total jarak, waktu, dan biaya dari sebuah rute.
    Ini menggantikan baris kode 'sum()' yang kompleks.
    """
    total_distance, total_time, total_cost = 0, 0, 0

    # Looping untuk setiap segmen dalam rute (contoh: Lampung -> Palembang)
    for i in range(len(path) - 1):
        start_node = path[i]
        end_node = path[i+1]
        
        # Cari data perjalanan untuk segmen ini di dalam graph
        for neighbor, data in graph[start_node]:
            if neighbor == end_node:
                total_distance += data['distance']
                total_time += data['time']
                total_cost += data['cost']
                break # Lanjut ke segmen berikutnya jika sudah ketemu
                
    return total_distance, total_time, total_cost

def print_route_analysis(title, path, total_dist, total_time, total_cost):
    """
    Fungsi untuk mencetak hasil analisis dengan format yang rapi.
    Ini untuk menghindari pengulangan kode print.
    """
    print(f"✅ {title}")
    print(f"   Jalur             : {' -> '.join(path)}")
    print(f"   Total Jarak       : {total_dist:.2f} km")
    print(f"   Total Waktu       : {total_time:.2f} jam")
    print(f"   Estimasi Biaya Tol: Rp {int(total_cost):,}.000")


# =====================================================================
# BAGIAN 2: BLOK EKSEKUSI UTAMA (Gantikan blok kode lama Anda dengan ini)
# =====================================================================

# Ganti 'create_graph_from_csv' dengan nama fungsi yang Anda gunakan jika berbeda
file_path = 'lampungmedan.csv' # Pastikan nama file ini sesuai
graph = create_graph_from_csv(file_path)

# Asumsikan 'graph' sudah ada dari fungsi sebelumnya
if graph:
    start_city = 'Lampung'
    goal_city = 'Medan'

    print("\n" + "="*50)
    print(f"Mencari rute optimal dari {start_city} ke {goal_city}...")
    print("="*50 + "\n")

    # --- 1. Analisis Rute Tercepat ---
    fastest_path, _ = a_star_search(graph, start_city, goal_city, cost_type='time')
    
    if fastest_path != "Rute tidak ditemukan":
        # Gunakan fungsi bantuan untuk menghitung detail dan mencetak hasil
        dist, time, cost = get_path_details(fastest_path, graph)
        print_route_analysis(
            "Rute TERCEPAT (Berdasarkan Waktu Tempuh)", 
            fastest_path, dist, time, cost
        )
    else:
        print("Rute tercepat tidak ditemukan.")

    print("\n" + "-" * 40 + "\n")

    # --- 2. Analisis Rute Termurah ---
    cheapest_path, _ = a_star_search(graph, start_city, goal_city, cost_type='cost')

    if cheapest_path != "Rute tidak ditemukan":
        # Gunakan fungsi bantuan yang sama
        dist, time, cost = get_path_details(cheapest_path, graph)
        print_route_analysis(
            "Rute TERMURAH (Berdasarkan Biaya Tol)",
            cheapest_path, dist, time, cost
        )
    else:
        print("Rute termurah tidak ditemukan.")

✅ Berhasil memuat data dari CSV dan membuat graph.

Mencari rute optimal dari Lampung ke Medan...

✅ Rute TERCEPAT (Berdasarkan Waktu Tempuh)
   Jalur             : Lampung -> Palembang -> Jambi -> Pekanbaru -> Rantau Prapat -> Medan
   Total Jarak       : 1690.00 km
   Total Waktu       : 25.70 jam
   Estimasi Biaya Tol: Rp 637.000

----------------------------------------

✅ Rute TERMURAH (Berdasarkan Biaya Tol)
   Jalur             : Lampung -> Baturaja -> Lubuk Linggau -> Bukittinggi -> Padang Sidempuan -> Medan
   Total Jarak       : 1435.00 km
   Total Waktu       : 28.70 jam
   Estimasi Biaya Tol: Rp 0.000
